In [ ]:
! pip install sigopt

In [ ]:
import torch
import torch.nn as nn
import torchvision
from torch.profiler import profile, record_function, ProfilerActivity

import numpy as np
from skimage import transform

import sigopt
import os

import time

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd

In [ ]:
os.environ["SIGOPT_API_TOKEN"] = ' insert SigOPT API Token'
%load_ext sigopt

In [ ]:
def get_inference_time(model, input):
  "Estimate inference time"

  Time = []

  time_start = time.time()

  _ = model(input)

  time_end = time.time()

  return time_end - time_start

In [ ]:
def profile_model(model, input):
  "Profile model"

  with profile(activities=[ProfilerActivity.CPU],
               profile_memory=True,
              with_flops=True,
               record_shapes=True) as prof:

    with torch.no_grad():
      model.forward(input)

  return [((e.cpu_memory_usage / 1024.0) / 1024) for e in prof.events() if e.name == 'aten::conv2d'][0]

In [ ]:
def get_model_size(model):
  "get the size of the stored model in mB"

  torch.save(model.state_dict(), "tmp.pt") 
  size_mb = os.path.getsize("tmp.pt")/1e6
  os.remove('tmp.pt')

  return size_mb


In [ ]:
experiment = sigopt.create_experiment(
  name="Peak memory",

  type="random",

  parameters=[     
    dict(name="in_channels_", type="int", bounds=dict(min=1, max=20)),
    dict(name="out_channels_", type="int", bounds=dict(min=1, max=20)),
    dict(name="kernel_size_", type="int", bounds=dict(min=1, max=20)),
    dict(name="stride_", type="int", bounds=dict(min=1, max=20)),
    dict(name="padding_", type="int", bounds=dict(min=1, max=20)),
    dict(name="batch_size_", type="int", bounds=dict(min=1, max=20)),

    dict(name="image_resolution_", type="double", bounds=dict(min=0.5, max=1)), 

  ],
  metrics=[
    dict(name="peak_memory[mB]", strategy = 'optimize', objective="minimize"),
    dict(name="inference_time[sec]", strategy = 'optimize', objective="minimize"),
    dict(name="storage_memory[mB]", strategy = 'store', objective="minimize"),
  ],
  parallel_bandwidth=1,
  budget=400,
  )

In [ ]:
for run in experiment.loop():
  with run:
    
    "Set hyperparameters"
    in_channels_ = run.params.setdefault('in_channels_', 3)
    out_channels_ = run.params.setdefault('out_channels_', 10)
    kernel_size_ = run.params.setdefault('kernel_size_', 10)
    
    s = run.params.setdefault('stride_', 1)
    stride_ = (s, s)

    p = s = run.params.setdefault('padding_', 1)
    padding_ = (p, p)

    batch_size_ = run.params.setdefault('batch_size_', 10)

    image_resolution_ = run.params.setdefault('image_resolution_', 0.5)
    image_size_ = run.params.setdefault('image_size_', 224)

    x_image_dim, y_image_dim = np.ceil(image_resolution_ * image_size_).astype('int'), np.ceil(image_resolution_ * image_size_).astype('int')

    "Create Conv2d block"
    m = nn.Conv2d(in_channels = in_channels_,
                  out_channels = out_channels_,
                  kernel_size = kernel_size_,
                  stride = stride_,
                  padding = padding_)
    
    "Create a random input"
    input = torch.randn(batch_size_,
                        in_channels_,
                        x_image_dim,
                        y_image_dim)

    run.log_metric("peak_memory[mB]", profile_model(model = m, input = input))
    run.log_metric("inference_time[sec]", get_inference_time(model = m, input = input))
    run.log_metric("storage_memory[mB]", get_model_size(model = m))

In [ ]:



experiment_id = " insert experiment ID "

In [ ]:
data = pd.DataFrame([], columns=["in_channels_",
                                 "out_channels_",
                                 "kernel_size_",
                                 "stride_",
                                 "padding_",
                                 "batch_size_",
                                 "image_resolution_",
                                 "peak_memory[mB]",
                                 "inference_time[sec]",
                                 "storage_memory[mB]",
                                 ])

for run in sigopt.get_experiment(experiment_id=experiment_id).get_runs():
  
  tmp = run.assignments

  tmp[run.values['peak_memory[mB]'].name] = run.values['peak_memory[mB]'].value

  tmp[run.values['inference_time[sec]'].name] = run.values['inference_time[sec]'].value

  tmp[run.values['storage_memory[mB]'].name] = run.values['storage_memory[mB]'].value


  data = data.append(tmp, ignore_index=True)

data['input_size_'] = np.ceil(data['image_resolution_'] * data['image_size_'])

data.drop(columns=['image_size_', 'image_resolution_'], inplace=True)

data.head()

In [ ]:
parameters = data[["in_channels_",
                   "out_channels_",
                   "kernel_size_",
                   "stride_",
                   "padding_",
                   "batch_size_",
                   "input_size_",]]


parameters.columns = ["Input Channels",
                      "Output Channels",
                      "Kernel Size",
                      "Stride",
                      "Padding",
                      "Batch Size",
                      "Input Size",]


metrics = data[['inference_time[sec]',
                'peak_memory[mB]',
                'storage_memory[mB]',]]

metrics_log = np.log(metrics)

metrics_log.columns = ['log(Inference Time)',
                       'log(Peak Memory)',
                       'log(Storage Memory)']

data_log = pd.concat([parameters, metrics_log], axis=1)

In [ ]:
con_inference_time = metrics['inference_time[sec]'].quantile(0.25)
con_peak_memory = metrics['peak_memory[mB]'].quantile(0.25)
con_storage_memory = metrics['storage_memory[mB]'].quantile(0.25)

hue = pd.DataFrame([(metrics['storage_memory[mB]'] < con_storage_memory),
                  (metrics['peak_memory[mB]'] < con_peak_memory),
                  (metrics['storage_memory[mB]'] < con_storage_memory)]).all()

hue.name = 'Constraints'

hue[hue == False] = 'Outside'
hue[hue == True] = 'Inside'

hue_order = hue.sort_values(ascending=False)

In [ ]:
sns.set_theme(style="ticks")
plot_metrics = sns.pairplot(metrics_log,
                            diag_kind="kde")

plot_metrics.fig.suptitle('Pair Plot of Application Specific Metrics',
                          y=1.04,
                          fontsize = 25)

In [ ]:
tmp = pd.concat([metrics_log, hue], axis=1)
tmp = tmp.sort_values(by = 'Constraints', ascending=False)

sns.set_theme(style="ticks")

plot_metrics = sns.pairplot(tmp,
                            hue='Constraints')

plot_metrics.fig.suptitle('Pair Plot of Application Specific Metrics',
                          y=1.04,
                          fontsize = 25)

plot_metrics._legend.remove()

In [ ]:

sns.set_theme(style="ticks")

plot_all = sns.pairplot(data_log,
                        diag_kind="kde",
                        y_vars=list(metrics_log),
                        x_vars=list(data_log))

plot_all.fig.suptitle('Pair Plot of Application Specific Metrics Including Relevant Hyperparameters',
                       y=1.04,
                       fontsize = 25)


In [ ]:
tmp = pd.concat([data_log, hue], axis=1)
tmp = tmp.sort_values(by = 'Constraints', ascending=False)

sns.set_theme(style="ticks")

plot_all = sns.pairplot(tmp,
                        diag_kind="kde",
                        hue = 'Constraints',
                        y_vars=list(metrics_log),
                        x_vars=list(data_log))

plot_all.fig.suptitle('Pair Plot of Application Specific Metrics Including Relevant Hyperparameters',
                       y=1.04,
                       fontsize = 25)

plot_all._legend.remove()